In [24]:
import pathlib

import tensorflow as tf
import xarray as xr
import numpy as np

In [11]:
data_dir = pathlib.Path('~/data/undeepwaves/gebco').expanduser()
paths = list(
    data_dir.glob('*/results/*.nc')
)

In [85]:
fr_paths = [path for path in paths if '-fr' in path.name]
hs_paths = [path for path in paths if not  '-fr' in path.name]

In [87]:
ds = xr.open_mfdataset(hs_paths, concat_dim='run', combine='nested')
ds = xr.open_dataset(hs_paths[1])

AttributeError: Tensor.name is meaningless when eager execution is enabled.

In [39]:
ds

<xarray.Dataset>
Dimensions:  (run: 1, x: 256, y: 256)
Coordinates:
  * run      (run) int32 1
  * x        (x) float32 0.0 10.039216 20.078432 ... 2539.9216 2549.961 2560.0
  * y        (y) float32 0.0 10.039216 20.078432 ... 2539.9216 2549.961 2560.0
Data variables:
    botl     (run, y, x) float32 ...
    hs       (run, y, x) float32 15.732291 15.684225 ... 18.554491 18.554491
    tps      (run, y, x) float32 ...
    tp       (run, y, x) float32 ...
    tmm10    (run, y, x) float32 ...
    tm01     (run, y, x) float32 ...
    tm02     (run, y, x) float32 ...
    theta0   (run, y, x) float32 ...
    spread   (run, y, x) float32 ...
    dhs      (run, y, x) float32 ...
    dtm      (run, y, x) float32 ...
    ssh      (run, y, x) float32 ...
    xwnd     (run, y, x) float32 ...
    ywnd     (run, y, x) float32 ...
    L        (run, y, x) float32 ...
Attributes:
    Conventions:             CF-1.5
    History:                 Created with agioncmd version 1.5
    Directional_convention:  nautical
    project:                 emulator
    run:

In [92]:
spec = {
    "variables": ['botl', 'hs', 'tm01'],
    "directional_variables": ['theta0'],
    "derived": ['mask']
}


def read_nc(paths=hs_paths):
    for path in paths:
        ds = xr.open_dataset(path)
        
        tensors = []
        
        mask = np.zeros((256, 256), dtype='int32')
        
        for var in spec['variables']:
            arr = np.squeeze(ds[var].values)
            mask = np.logical_or(mask, np.isnan(arr))
            arr = np.nan_to_num(arr)
            tensor = tf.convert_to_tensor(arr)
            tensors.append(tensor)
        
        for var in spec['directional_variables']:
            arr = np.squeeze(ds[var].values)
            mask = np.logical_or(mask, np.isnan(arr))
            arr = np.nan_to_num(arr)
            # convert to rad
            if ds[var].units == 'degrees':
                arr = np.deg2rad(arr)
            tensor_x = tf.convert_to_tensor(np.cos(arr))
            tensor_y = tf.convert_to_tensor(np.sin(arr))
            tensors.append(tensor_x)
            tensors.append(tensor_y)

        tensor_mask = tf.convert_to_tensor(mask)
        tensors.append(mask)
            
        
        
        yield tuple(tensors)
    
    
dataset = tf.data.Dataset.from_generator(
    read_nc, 
    output_signature=(
         tf.TensorSpec(shape=(256, 256), dtype=tf.float32),
         tf.TensorSpec(shape=(256, 256), dtype=tf.float32),
         tf.TensorSpec(shape=(256, 256), dtype=tf.float32),
         tf.TensorSpec(shape=(256, 256), dtype=tf.float32),
         tf.TensorSpec(shape=(256, 256), dtype=tf.float32),
         tf.TensorSpec(shape=(256, 256), dtype=tf.int32)
    )
)


In [103]:
for example in dataset.shuffle(10):
    print(example)
    break


(<tf.Tensor: shape=(256, 256), dtype=float32, numpy=
array([[ 2.0411334,  2.0411334,  2.0411334, ...,  3.0411334,  3.0411334,
         3.0411334],
       [ 2.0411334,  2.0411334,  2.0411334, ...,  3.0411334,  3.0411334,
         3.0411334],
       [ 2.0411334,  2.0411334,  2.0411336, ...,  3.0411334,  3.0411334,
         3.0411334],
       ...,
       [ 1.0411334,  1.0411335,  1.0411334, ..., -1.3604352, -1.5596539,
        -1.560435 ],
       [ 1.0411334,  1.0411334,  1.0411334, ..., -1.7573041, -1.7580823,
        -1.7580853],
       [ 1.0411334,  1.0411334,  1.0411334, ..., -1.7588665, -1.7588665,
        -1.7588665]], dtype=float32)>, <tf.Tensor: shape=(256, 256), dtype=float32, numpy=
array([[6.8941307, 6.9071026, 6.92465  , ..., 6.4035454, 6.4371166,
        6.4500866],
       [6.890316 , 6.902523 , 6.9193096, ..., 6.4035454, 6.434828 ,
        6.444746 ],
       [6.8857384, 6.8964195, 6.912443 , ..., 6.3921013, 6.4233837,
        6.43025  ],
       ...,
       [8.10648  , 8.0988